In [15]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sn

import healpy as hp
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles
import lsst.sims.maf.db as db
import lsst.sims.maf.plots as plots




In [36]:
import lsst.sims.maf.maps as maps

In [5]:
plt.rc('legend', fontsize=20) # using a size in points
plt.rc('xtick', labelsize=16)
plt.rc('ytick', labelsize=16)
plt.rcParams['axes.labelsize'] = 16

In [6]:
# load opsim database
dbpath_v15 = "/home/idies/workspace/lsst_cadence/FBS_1.5/"  # path to all opsim databases

dbpath_v17 = "/home/idies/workspace/lsst_cadence/FBS_1.7/"

dbpath_v171 = "/home/idies/workspace/lsst_cadence/FBS_1.7.1/"


# output directory
dataRawDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/tGaps/'

outDir = '/home/idies/workspace/Temporary/lixl/scratch/outDir/resultsDb/'
resultsDb = db.ResultsDb(outDir=outDir)



In [7]:
# get the name of all opsim dbs 
import glob

workpath = os.getcwd()
#workpath = '/home/idies/workspace/Storage/lixl/persistent/LSST_OpSim/unknowns'

os.chdir(dbpath_v15)  # change to opsim database directory
dblist_v15 = glob.glob('*.db') 

os.chdir(dbpath_v17)  # change to opsim database directory
dblist_v17 = glob.glob('*.db') 

os.chdir(dbpath_v171)  # change to opsim database directory
dblist_v171 = glob.glob('*.db') 

os.chdir(workpath) # change back to work directory

dblist_v15.sort()
dblist_v17.sort()
dblist_v171.sort()




In [34]:
from scipy.interpolate import interp1d

#__all__ = ['StarDensityMetric']

class StarDensityMetric(metrics.BaseMetric):
    """Interpolate the stellar luminosity function to return the number of
    stars per square arcsecond brighter than the rmagLimit. Note that the
    map is built from CatSim stars in the range 20 < r < 28."""

    def __init__(self, rmagLimit=25., units='stars/sq arcsec', maps=['StellarDensityMap'], **kwargs):

        super(StarDensityMetric, self).__init__(col=[], 
                                                maps=maps, units=units, **kwargs)
        self.rmagLimit = rmagLimit

    def run(self, dataSlice, slicePoint=None):
        # Interpolate the data to the requested mag
        interp = interp1d(slicePoint['starMapBins'][1:], slicePoint['starLumFunc'])
        # convert from stars/sq degree to stars/sq arcsec
        starDens = interp(self.rmagLimit)/(3600.**2)
        
        result = {}
        result['starDens'] = starDens
        result['slicePoint'] = slicePoint
        
        return starDens
    
#def reduce_star_count(self, metric):
#    return metric['starDens']
    


In [37]:
stellardensitymap = maps.StellarDensityMap(filtername = 'g')

In [42]:
metric = StarDensityMetric()

sqlstr ='proposalID =1 and note not like  "DD%" and night< 720'
slicer = slicers.HealpixSlicer(nside=16) 

# bundle
newMetricSky = metricBundles.MetricBundle(metric, slicer, sqlconstraint, mapsList=[stellardensitymap])


dbpath = dbpath_v15
dbname = dblist_v15[-1]
opsdb = db.OpsimDatabase(dbpath+dbname)

# group bundle
bundleDict = {'newMetricSky':newMetricSky}
group = metricBundles.MetricBundleGroup(bundleDict, opsdb, outDir=outDir, resultsDb=resultsDb)

# run
group.runAll()
#group.plotAll(closefigs = False)



Healpix slicer using NSIDE=16, approximate resolution 219.871130 arcminutes
Querying database SummaryAllProps with constraint proposalID =1 and note not like  "DD%" and night< 720 for columns ['fieldRA', 'fieldDec']
Found 354605 visits
Running:  ['newMetricSky']


KeyError: 'starMapBins'

In [21]:
dblist_v15[-1]

'wfd_depth_scale0.99_v1.5_10yrs.db'